In [1]:
import time
#import board
#import busio
import pandas as pd
from pandas import DataFrame
from matplotlib import pylab
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as clrs
import matplotlib.cm as cm
import matplotlib.dates as mdates
import matplotlib
import seaborn as sns
import datetime
import os
from pandas import concat
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.regularizers import l2
from numpy import loadtxt
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [5]:
def file():
    
    #path = '/home/yogender/Desktop/KaggleHousePricePrediction/raspiData/Raspi2_second_currentLoop_RemoteMySql'
    #file_name = 'csvFile_2021_04_20'
    #file_name = 'csvFile_2021_04_28'
    #file_name = 'csvFile_2021_04_28'
    file_name = 'csvFile_2021_05_24'
    #file_name = 'DLTemLongerDatasetDischargecsvFile_2021_05_20'
    #file_name = 'csvFile_2021_04_19'
    #file_name ='csvFile_2021_03_30'
    suffix = '.csv'
    filePath = os.path.join(os.getcwd(), file_name + suffix)
    return  filePath


In [8]:
def fileNameForDLFigures(midfix = None):
    
    
    from datetime import date
    today = date.today()
    # dd/mm/YY
    d1 = today.strftime("%d_%m_%Y")
    #print("d1 =", d1)
    cwdir = os.getcwd()
    mkSubDir =  os.path.join(cwdir, d1)
    mkSubDir
    if not os.path.exists(mkSubDir):
        #os.makedirs('my_folder')
        os.mkdir(mkSubDir)
    #path = '/home/yogender/Desktop/KaggleHousePricePrediction/raspiData/Raspi2_second_currentLoop_RemoteMySql'
    #file_name = 'csvFile_2021_04_20'
    head, tail = os.path.split(file())
    name =tail.split('_')[1:]
    file_name = "_".join(name)
    file_name=file_name.split('.')[0]
    #join

    #file_name = 'DL_2021_04_28'
    #file_name = 'csvFile_2021_04_19'
    #file_name ='csvFile_2021_03_30'
    
    #cwdir = os.getcwd()
    #mkSubDir =  os.path.join(cwdir, 'DL_sub')
    suffix = '.svg'
    filePath = os.path.join(mkSubDir, midfix + file_name +suffix)
    return  filePath

In [9]:
fileNameForDLFigures(midfix = 'test')

'/home/yogender/Desktop/KaggleHousePricePrediction/raspiData/Raspi2_second_currentLoop_RemoteMySql/31_05_2021/test2021_05_24.svg'

In [10]:
def modelDir(ModelName=None):
    model = fileNameForDLFigures(midfix = 'Tem').split('/')[:-1]
    model_dir = "/".join(model)
    #model_dir 
    modelPath = os.path.join(model_dir, ModelName)
    return modelPath

In [11]:
def filePath2Pandas(file_path):
    drop_column = 'Unnamed: 0'
    df = pd.read_csv(file_path, delimiter=',').drop(drop_column, axis = 'columns')

    column_names =  ['id', 'dateTime', 'Tamb', 'TtopTestTankHPCir', 'TbottomTestTankHpCir',
       'TtopSourceTank', 'TloadTankMix', 'TTopTestTankLoadCir', 'TloadMix',
       'TbottomSourceTank', 'TbottomTestTankLoadCir', 'T4', 'T3', 'T2', 'T1',
        'T0', 'T9', 'T8', 'T7', 'T6', 'T5', 'flowHP', 'flowLoad', 'Load_kW',
       'Heat_Capacity_kW', 'electrcity_kW', 'COP', 'nDis', 'timeStamp_Temp',
       'timeStamp_watt', 'entropy_kJ/K', 'entropy_kJ', 'nStratCharging',
       'TotalTanksEnergyForAvailability_kJ', 'DisEntropy_kj/K',
       'DisEntropy_kj', 'nStratDischarge',
       'DisTotalTanksEnergyForAvailability_kJ']
    df.columns =column_names
    df = df[['id', 'dateTime', 'Tamb', 'TtopTestTankHPCir', 'TbottomTestTankHpCir',
       'TtopSourceTank', 'TloadTankMix', 'TTopTestTankLoadCir', 'TloadMix',
       'TbottomSourceTank', 'TbottomTestTankLoadCir', 'T0', 'T1', 'T2', 'T3',
        'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'flowHP', 'flowLoad', 'Load_kW',
       'Heat_Capacity_kW', 'electrcity_kW', 'COP', 'nDis', 'timeStamp_Temp',
       'timeStamp_watt', 'entropy_kJ/K', 'entropy_kJ', 'nStratCharging',
       'TotalTanksEnergyForAvailability_kJ', 'DisEntropy_kj/K',
       'DisEntropy_kj', 'nStratDischarge',
       'DisTotalTanksEnergyForAvailability_kJ']]
    
    #df.columns.values[[11,12,13,14,15,16,17,18,19,20]] = ['T0', 'T1', 'T2', 'T3','T4', 'T5', 'T6', 'T7', 'T8', 'T9']

    return df
df_gross=filePath2Pandas(file())  


In [13]:
gross_tem = df_gross[['T0', 'T1', 'T2', 'T3','T4', 'T5', 'T6', 'T7', 'T8', 'T9']]

In [128]:
gross_tem_two = df_gross[['T0', 'T1', 'T3']]
gross_tem_two

,T0,T1,T3
0,22.6232,23.0288,23.8403
1,22.6232,22.9950,23.8064
2,22.6232,23.0288,23.8064
3,22.6232,23.0288,23.8064
4,22.6232,23.0288,23.7726
...,...,...,...
1291,22.2852,24.2461,24.4828
1292,22.2852,24.2461,24.4828
1293,22.2852,24.2799,24.4828
1294,22.2852,24.2799,24.4828


In [129]:
list_x = [gross_tem_two.iloc[:, i] for i in range (gross_tem_two.shape[1])]
df_1 = pd.DataFrame(list_x[0])
#list_one = np.array(list_x[0]).reshape(1,-1)

scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(df_1)

In [57]:
type(list_x[0])
list_one = np.array(list_x[0])
list_one.reshape(1,-1).shape

(1, 1296)

In [130]:
"""
Normalization should be done BEFORE one prepares the data for ANN or LSTM input.
Normalization is done feature-wise (that means column-wise).
"""

def normalize(X):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler = scaler.fit(X)
    scaled_data = scaler.transform(X)
    return scaled_data, scaler

'''
df_nrm_dis, scaler = normalize(gross_tem_oneValue)
df_nrm_dis = pd.DataFrame(df_nrm_dis)
#df_nrm.columns = chargingIn_df.columns

df_nrm_dis.columns = [ 'T6']
df_nrm_dis

'''

"\ndf_nrm_dis, scaler = normalize(gross_tem_oneValue)\ndf_nrm_dis = pd.DataFrame(df_nrm_dis)\n#df_nrm.columns = chargingIn_df.columns\n\ndf_nrm_dis.columns = [ 'T6']\ndf_nrm_dis\n\n"

In [131]:
def nameEachColumn(df, i):
    columnName  = [f"T{i}"] 
    df.columns =  columnName
    return df


In [142]:
def loopOverFunction(df):
    tuple_arr_nrm_dis= [normalize(pd.DataFrame(df.iloc[:,i])) for i in range (df.shape[1])]
    df_nrm_dis = [pd.DataFrame(i[0]) for i in tuple_arr_nrm_dis]
    #columnName  = [f"T{i}" for i in range (df.shape[1])]
    df_nrm_dis = [nameEachColumn(df, i) for i,df in enumerate (df_nrm_dis) ]
    scalers = [scaler[1] for scaler in tuple_arr_nrm_dis]
    
    return df_nrm_dis, scalers 
#loopOverFunction(gross_tem_two)

In [139]:
k =3
n_features = 1
#out_features = y_train.shape[1]
#out_features
epochs =5000
batch_size = 500

In [144]:
def flatten_row_wise(df):
    """Take row by row and attach to one flat single row."""
    #arr = np.ndarray.flatten(np.array(df))
    #append(arr,)
    
    return np.ndarray.flatten(np.array(df))

def prepare_df(df):
    n_rows, n_cols = df.shape
    new_rows = np.array([flatten_row_wise(df.iloc[(i-k):i]) for i in range(k, n_rows)])
    new_ys = np.array([row for row in df.iloc[(k):, :].itertuples(index=False)])
    # idxs = [x[0] for x in new_ys]
    # new_ys = [x[1] for x in new_ys]
    return new_rows, new_ys

In [167]:
def prepare_list_of_df(tuple_df):
    l_df = tuple_df[0]
    list_of_tuple_Xdf1_ydf1 = [prepare_df(df) for df in l_df] 
    list_Xdf1 = [i[0] for i in list_of_tuple_Xdf1_ydf1]
    list_ydf1 = [i[1] for i in list_of_tuple_Xdf1_ydf1]
    #x = [(i[0].shape, i[1]) for i in list_of_tuple_Xdf1]
    #print (list_ydf1[0])
    return list_Xdf1, list_ydf1
#type(prepare_list_of_df(loopOverFunction(gross_tem_two)))

In [170]:
def train_test_split_indexes(X, y, test_size=0.2, random_state=42, shuffle=False):
    X_train_indexes, X_test_indexes, y_train_indexes, y_test_indexes = train_test_split(pd.DataFrame(list(range(X.shape[0]))),
                                                                                        pd.DataFrame(list(range(y.shape[0]))),
                                                                                        test_size=test_size,
                                                                                        random_state=random_state,
                                                                                        shuffle=shuffle)
    train_indexes, test_indexes = [x for x in X_train_indexes.iloc[:, 0]]  , [x for x in X_test_indexes.iloc[:, 0]]
    return sorted(train_indexes), sorted(test_indexes)

In [169]:
def list_train_test_indexes (tuple_df):
    l_Xdf1 = tuple_df[0]
    l_ydf1 = tuple_df[1]
    
    
    return l_ydf1
list_train_test_indexes (prepare_list_of_df(loopOverFunction(gross_tem_two)))

[array([[0.06567129],
        [0.06567129],
        [0.06567129],
        [0.06567129],
        [0.06567129],
        [0.06567129],
        [0.06778992],
        [0.06567129],
        [0.06567129],
        [0.06567129],
        [0.06567129],
        [0.06567129],
        [0.06567129],
        [0.06567129],
        [0.06567129],
        [0.06567129],
        [0.06567129],
        [0.06567129],
        [0.06567129],
        [0.06567129],
        [0.06567129],
        [0.06567129],
        [0.06355266],
        [0.06567129],
        [0.06567129],
        [0.06567129],
        [0.06778992],
        [0.06778992],
        [0.06567129],
        [0.06778992],
        [0.06990855],
        [0.06990855],
        [0.06990855],
        [0.06778992],
        [0.06990855],
        [0.07202718],
        [0.07414581],
        [0.07414581],
        [0.07626444],
        [0.07626444],
        [0.0805017 ],
        [0.0805017 ],
        [0.08473896],
        [0.08262033],
        [0.08473896],
        [0